In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from sys import argv
from os.path import exists
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import selenium
import re
import requests

Defining a function to scrape the data
=============

The function below scrapes stock data from gurufocus. As of now for each stock symbol it goes to one webpage and saves the Graham number and current stock price and returns the data in a list of lists.

Future goals include extending the function to take two inputs; ticker and a list. The list contains [xpath name, and base URL]. The function will then return a list of lists, with each sublist being ticker any other data assciated to n-xpaths in the list.

-------------------------------------------------------------------------------------------------------------------------------
gnumber = sqrt of (22.5	x	Tangible Book Value per Share	x	EPS without Non-Recurring Items (NRI) (TTM))

Peter Lynch fair value [P/E (NRI) Ratio	/	EBITDA Growth Rate (5-year average) ]x 5-Year EBITDA Growth Rate	x	Earnings Per Share (NRI)

In [34]:
def GuruFocusScrape(tickerName, scraped_list):

    """
    Function opens gurufocus.com for a given tickerName and returns some value.
    The function needs three inputs, tickerName, baseWebpage, XPath element list
    """
    
    baseWebpage = "https://www.gurufocus.com/term/grahamnumber/" + tickerName + "/Graham%2BNumber/" + tickerName
    #print(baseWebpage)

    request = requests.get(baseWebpage)
    #print(request)
    if request.status_code == 200:
    
        driver = webdriver.Chrome() # Open chrome browser
        wait = WebDriverWait(driver, 60) #defines an object that allows an expected condition to call it

        driver.get(baseWebpage)
        #driver.get(baseWebpage)
        
        if baseWebpage == driver.current_url:
        
            elementXPath = '//*[@id="cboxClose"]' # identifier button that closes stupid pop-up

            # EC.elment_to... retuns a value of thing that does or doesn't exist, wait.until waits 60s until EC returns True
            element = wait.until(EC.element_to_be_clickable((By.XPATH,elementXPath)))
            element.click() #clicks element when it becomes clickable

            #gnumberXPath = '//*[@id="div_right"]/div[3]'
            gnumberXPath = '//*[@id="div_right"]/p[2]/strong[5]'
            priceXPath = '//*[@id="stock_profile_top"]/div[3]/font[1]'

            try: #gnumber = driver.find_element_by_xpath(gnumberXpath).text:
                
                gnumber = driver.find_element_by_xpath(gnumberXPath).text
                price = driver.find_element_by_xpath(priceXPath).text
            except:
                gnumber = 'NA'
                price = 'NA'
        
        else:

            gnumber = 'NA'
            price = 'NA'
    
    else:
        
        gnumber = 'NA'
        price = 'NA'
    
    
    sleep(5)   
    driver.close()
    print([tickerName, gnumber, price, baseWebpage])
    return scraped_list.append([tickerName, gnumber, price])

 

Stock Tickers:
========
Obtaining all the nasdaq listed stocks for a given day from the nasdaq website. The list of companies that are traded on the index is given by nasdaq as a text file with delimter '|' and updated daily.

In [35]:
# Assign url of file: url
url = 'ftp://ftp.nasdaqtrader.com/symboldirectory/nasdaqlisted.txt'

# Read file into a DataFrame: df
nasdaq_info = pd.read_csv(url, sep='|')
tickers_to_get = nasdaq_info.iloc[29:50,0]

Obtaining, formatting, and cleaning the data
===========
Call the function with all stock tickers of interest, return a list, then format list to be readable df. Merge original nasdaq df and our new df. Finally filter the list for interested criteria.

In [36]:

ScrapedList =[]

#[GuruFocusScrape(ticker) for ticker in df['Symbol']] #loops for entire column of stock symbols, some 10,000
GNumber_list = [GuruFocusScrape(ticker, ScrapedList) for ticker in tickers_to_get] #loops over defined number of stock symbols
##---- Might try using tickers_to_get.apply(GuruFocusScrape(ticker,Scrapelist), axis=1)


#Turn returned scraped data list into a df
ScrapedData_df = pd.DataFrame(ScrapedList)
ScrapedData_df.columns = (['Symbol', 'GNumber', 'Stock Price'])

##----Comment these
ScrapedData_df['Stock Price'] = pd.to_numeric(ScrapedData_df['Stock Price'].str[1::], errors='coerce')
ScrapedData_df['GNumber'] = pd.to_numeric(ScrapedData_df['GNumber'].str[1::], errors='coerce')

#merge nasdaq_df and ScrapedData_df together on column named Symbol
nasdaq_scraped_data_df = pd.merge(nasdaq_info, ScrapedData_df, on='Symbol')

##----Comment this
nasdaq_scraped_data_df.to_csv('C:\\Users\\RiggsSc\\Documents\\SCR\\GWest_data\\nasdaq_scraped_data.csv')

#print(ScrapedData_df)
print(nasdaq_scraped_data_df)

['ACFC', u'USD 7.35', u'$7.86', 'https://www.gurufocus.com/term/grahamnumber/ACFC/Graham%2BNumber/ACFC']
['ACGL', u'$80.07', u'$97.3', 'https://www.gurufocus.com/term/grahamnumber/ACGL/Graham%2BNumber/ACGL']
['ACGLP', 'NA', 'NA', 'https://www.gurufocus.com/term/grahamnumber/ACGLP/Graham%2BNumber/ACGLP']
['ACHC', u'$N/A', u'$50.67', 'https://www.gurufocus.com/term/grahamnumber/ACHC/Graham%2BNumber/ACHC']
['ACHN', u'$N/A', u'$4.32', 'https://www.gurufocus.com/term/grahamnumber/ACHN/Graham%2BNumber/ACHN']
['ACIA', u'$32.19', u'$42.21', 'https://www.gurufocus.com/term/grahamnumber/ACIA/Graham%2BNumber/ACIA']
['ACIU', u'$N/A', u'$8.17', 'https://www.gurufocus.com/term/grahamnumber/ACIU/Graham%2BNumber/ACIU']
['ACIW', u'$N/A', u'$23.91', 'https://www.gurufocus.com/term/grahamnumber/ACIW/Graham%2BNumber/ACIW']
['ACLS', u'$18.44', u'$23.2', 'https://www.gurufocus.com/term/grahamnumber/ACLS/Graham%2BNumber/ACLS']
['ACNB', u'$27.79', u'$28.6', 'https://www.gurufocus.com/term/grahamnumber/ACNB/Gr

In [37]:
#Here do all analysis and filter of data
print(nasdaq_scraped_data_df)

   Symbol                                      Security Name Market Category  \
0    ACFC  Atlantic Coast Financial Corporation - Common ...               G   
1    ACGL             Arch Capital Group Ltd. - Common Stock               Q   
2   ACGLP  Arch Capital Group Ltd. - Depositary Shares Re...               Q   
3    ACHC     Acadia Healthcare Company, Inc. - Common Stock               Q   
4    ACHN     Achillion Pharmaceuticals, Inc. - Common Stock               Q   
5    ACIA         Acacia Communications, Inc. - Common Stock               Q   
6    ACIU                        AC Immune SA - Common Stock               G   
7    ACIW                 ACI Worldwide, Inc. - Common Stock               Q   
8    ACLS          Axcelis Technologies, Inc. - Common Stock               Q   
9    ACNB                    ACNB Corporation - Common Stock               S   
10   ACOR           Acorda Therapeutics, Inc. - Common Stock               Q   
11   ACRS          Aclaris Therapeutics,

In [3]:
import requests

url ='http://api.wolframalpha.com/v2/query?appid=DEMO&input=population%20of%20france&output=json'
test = requests.get(url)
json_data = test.json()
print(test)

<Response [200]>


In [ ]:
#fit G. west with r^2 values
#will FB-prophet method df.make_future_forecast() help?